# Multi-Agent System Demo

This notebook demonstrates a **conceptual** multi-agent setup, where a **“Master Orchestrator”** delegates tasks to **three specialized sub-agents**:
1. **Reflexion Agent** – Handles **local data analysis** (generates code to process a CSV, self-corrects on errors).
2. **Tool-Using Agent** – Interfaces with **local knowledge bases or internal tools**.
3. **Auto-GPT Agent** – Performs **external web research** (visiting and scraping websites).

The **goal**:
 > “Perform an advanced analysis of EV (electric vehicle) data from a CSV, gather external info from websites, also consult internal knowledge base about EV manufacturing, then produce a final recommendation for the best EV to focus on.”

We split the logic into **five sections**:
1. **Reflexion Agent** (`reflexion_agent.py`)
2. **Tool-Using Agent** (`tool_using_agent.py`)
3. **Auto-GPT Agent** (`autogpt_agent.py`)
4. **Multi-Agent Orchestrator** (`multi_agent_orchestrator.py`)
5. **Main** (`main.py`)

**Note**: This is a **high-level illustration**. In production, each agent might have more detailed logic, memory structures, LLM calls, and robust error handling.

## Section 1: Reflexion Agent
A **Reflexion**-based CSV agent that:
1. Receives a user request.
2. "Generates" Python code to analyze a provided DataFrame.
3. Executes the code in a simple environment.
4. If there are errors, **reflects** and **corrects** the code.

For simplicity, we mock the code generation and correction steps.

In [ ]:
# reflexion_agent.py
import pandas as pd

class ReflexionCSVAgent:
    def __init__(self, max_reflections=2):
        self.max_reflections = max_reflections
        self.conversation_log = []

    def analyze_data(self, user_query, df):
        """
        Return analysis result (text or data structure). 
        In a real system, you'd generate code with an LLM.
        """
        self.conversation_log.append(f"User query for data analysis: {user_query}")

        # 1. Pretend to generate code.
        code_snippet = self.generate_code(user_query)
        success, outcome = self.run_generated_code(code_snippet, df)

        reflections = 0
        while not success and reflections < self.max_reflections:
            reflections += 1
            # 2. Reflect and correct
            corrected_code = self.reflect_and_correct(code_snippet)
            code_snippet = corrected_code
            success, outcome = self.run_generated_code(code_snippet, df)

        if success:
            return outcome
        else:
            return "Reflexion agent failed to produce a valid analysis result."

    def generate_code(self, query):
        # Mock code snippet
        code = (
            "# code snippet: filter EV data, get average range, etc.\n"
            "analysis_result = {\n"
            "  'max_range': df['Range'].max(),\n"
            "  'avg_price': df['Price'].mean()\n"
            "}\n"
            "result = analysis_result"
        )
        return code

    def run_generated_code(self, code_str, df):
        local_env = {"df": df, "result": None}
        try:
            exec(code_str, {}, local_env)
            return True, local_env["result"]
        except Exception as e:
            return False, str(e)

    def reflect_and_correct(self, original_code):
        # Example: fix a small bug by changing 'Range' to 'range'
        corrected_code = original_code.replace("df['Range']", "df['range']")
        return corrected_code


## Section 2: Tool-Using Agent
A **Tool-Using** agent that can **query an internal knowledge base** or call some local tool. Here, we simulate a minimal knowledge base with a dictionary.

In [ ]:
# tool_using_agent.py

MOCK_KNOWLEDGE_BASE = {
    "EV manufacturing": "Production capacity for new EV models is ramping up. "
                         "Batteries come from multiple suppliers with varying costs.",
    "Battery recycling": "Company policy emphasizes eco-friendly recycling programs."
}

class ToolUsingAgent:
    def __init__(self):
        self.conversation_log = []

    def query_internal_kb(self, topic):
        """
        Queries an internal knowledge base or specialized tool.
        """
        self.conversation_log.append(f"ToolUsingAgent received topic: {topic}")
        response = MOCK_KNOWLEDGE_BASE.get(topic, "No info available.")
        self.conversation_log.append(f"ToolUsingAgent answer: {response}")
        return response


## Section 3: Auto-GPT Agent
A **mini** Auto-GPT approach that searches external web data (mocked again). We define:
- `web_search_mock(topic)`: returns a list of relevant sites.
- `web_scrape_mock(url)`: returns some text from that site.

The agent’s `research_topic` method simulates collecting all relevant info from those sites.

In [ ]:
# autogpt_agent.py

MOCK_WEB_SITES = {
    "best_ev_reviews.com": "The most popular EV among reviewers is the Tesla Model Y.",
    "ev_price_tracker.com": "Current average price of Tesla Model Y is around $53,000."
}

def web_search_mock(topic):
    sites = []
    if "best ev" in topic.lower():
        sites.append("best_ev_reviews.com")
    if "price" in topic.lower():
        sites.append("ev_price_tracker.com")
    if not sites:
        sites.append("general_ev_site.com")
    return sites

def web_scrape_mock(url):
    return MOCK_WEB_SITES.get(url, "No relevant data found.")

class AutoGPTAgent:
    def __init__(self):
        self.visited_sites = set()
        self.conversation_log = []

    def research_topic(self, topic):
        """
        Iteratively 'plans' which sites to visit and merges data.
        We'll do a single pass for simplicity.
        """
        self.conversation_log.append(f"AutoGPTAgent: Searching web about '{topic}'")
        sites = web_search_mock(topic)
        results = []
        for s in sites:
            if s not in self.visited_sites:
                scraped = web_scrape_mock(s)
                results.append(scraped)
                self.visited_sites.add(s)
        # Return combined data
        return "\n".join(results)


## Section 4: Multi-Agent Orchestrator
This is the **master** agent that **coordinates** the three specialized agents:
1. Calls the **ReflexionCSVAgent** to analyze a local CSV.
2. Calls the **ToolUsingAgent** to query internal knowledge.
3. Calls the **AutoGPTAgent** for external research.
4. Merges everything into a final user-facing recommendation.

In a real system, you could have more dynamic logic, deciding which agent to call next based on partial outputs.

In [ ]:
# multi_agent_orchestrator.py

import pandas as pd
from reflexion_agent import ReflexionCSVAgent
from tool_using_agent import ToolUsingAgent
from autogpt_agent import AutoGPTAgent

class MultiAgentOrchestrator:
    def __init__(self):
        # Instantiate sub-agents
        self.reflexion_agent = ReflexionCSVAgent()
        self.tool_agent = ToolUsingAgent()
        self.autogpt_agent = AutoGPTAgent()

    def handle_request(self, user_query, df):
        """
        1) Use Reflexion Agent to analyze local CSV (EV data).
        2) Use Tool-Using Agent to gather internal knowledge.
        3) Use Auto-GPT Agent for external web data.
        4) Merge everything into a final recommendation.
        """
        # Step 1: Local CSV Analysis
        analysis_result = self.reflexion_agent.analyze_data(user_query, df)

        # Step 2: Get internal knowledge about EV manufacturing
        internal_info = self.tool_agent.query_internal_kb("EV manufacturing")

        # Step 3: External web research
        external_research = self.autogpt_agent.research_topic("Best EV in market and price")

        # Step 4: Combine results
        final_answer = self._combine_results(analysis_result, internal_info, external_research)
        return final_answer

    def _combine_results(self, analysis_result, internal_info, external_data):
        # Summarize all agent results
        if isinstance(analysis_result, dict):
            summary_text = (
                f"Local CSV Analysis:\n"
                f" - Max EV Range: {analysis_result.get('max_range', 'Unknown')}\n"
                f" - Average Price in CSV: {analysis_result.get('avg_price', 'Unknown')}\n\n"
            )
        else:
            summary_text = f"Local CSV Analysis: {analysis_result}\n\n"

        summary_text += f"Internal Knowledge: {internal_info}\n\n"
        summary_text += f"External Web Data:\n{external_data}\n\n"

        # A simplistic final recommendation
        recommendation = (
            "Based on local data, internal knowledge, and external reviews:\n"
            " - We found the top EV has the highest range from our local CSV.\n"
            " - External sources suggest Tesla Model Y is a leading choice.\n"
            " - Corporate knowledge indicates manufacturing is ramping up.\n"
            "Therefore, we recommend focusing on Tesla Model Y for further investment."
        )

        return summary_text + "\n" + recommendation


## Section 5: Main Orchestration (Entry Point)
Here, we put everything together:
- Create a **sample DataFrame** to simulate CSV data.
- Define a **user query** that requests advanced analysis.
- Instantiate the **MultiAgentOrchestrator**.
- Call `handle_request(...)`.
The orchestrator coordinates all sub-agents, returning a consolidated response.


In [ ]:
# main.py
import pandas as pd
from multi_agent_orchestrator import MultiAgentOrchestrator

if __name__ == "__main__":
    # Create a sample DataFrame (simulating our local EV CSV)
    sample_data = {
        "Model": ["EV A", "EV B", "Tesla Model Y", "EV C"],
        "Range": [250, 220, 300, 280],
        "Price": [40000, 35000, 50000, 45000]
    }
    df = pd.DataFrame(sample_data)

    # High-level user query
    user_query = (
        "Perform an advanced analysis of EV data from CSV, gather external info, "
        "and also check corporate knowledge about EV manufacturing. Provide a recommendation."
    )

    orchestrator = MultiAgentOrchestrator()
    final_response = orchestrator.handle_request(user_query, df)

    print("=== FINAL RESPONSE ===")
    print(final_response)


## How to Run
1. **Run all cells** in this notebook.
2. The last cell simulates `main.py` and prints the final response.
3. You may see something like:
   ```
   === FINAL RESPONSE ===
   Local CSV Analysis:
    - Max EV Range: 300
    - Average Price in CSV: 42500.0

   Internal Knowledge: Production capacity for new EV models is ramping up. Batteries come from multiple suppliers...

   External Web Data:
   The most popular EV among reviewers is the Tesla Model Y.
   Current average price of Tesla Model Y is around $53,000.

   Based on local data, internal knowledge, and external reviews:
    - We found the top EV has the highest range from our local CSV.
    - External sources suggest Tesla Model Y is a leading choice.
    - Corporate knowledge indicates manufacturing is ramping up.
   Therefore, we recommend focusing on Tesla Model Y for further investment.
   ```

## Key Points
1. **Multi-Agent**: Three specialized sub-agents for different tasks.
2. **Master Orchestrator**: Manages the workflow, calling each agent in sequence.
3. **Reflexion**: Used for local CSV analysis (generates and executes code, self-corrects on errors).
4. **Tool-Using**: Queries a local knowledge base.
5. **Auto-GPT**: Performs external web research (mocked here but demonstrates the concept).
6. **Final Consolidation**: Orchestrator merges all sub-results into one comprehensive recommendation.

This demonstration is intentionally **conceptual**—in real-world usage, each agent could:
- Use actual LLM calls.
- Have more complex memory or knowledge retrieval.
- Follow advanced error handling or collaboration protocols.

Still, it shows how you can **compose** multiple specialized agent patterns under one **multi-agent** umbrella to address a **high-level user request** in a structured, orchestrated manner.